# `erfindungen_extract_data.ipynb` 

This file extracts the data from the "Erfindungen" overview of patents until 1852 in the Austro-Hungarian empire. The .pdf already contains implicit text data, however, OpenAI Chat Completions cannot handle this very well. Hence, we opted to extract the text manually using the `tabula` piece of software. 

See [here](https://tabula.technology/) how to install tabula. It's a rule based approach based on either row finding or column finding. The base file that I use has column finding. This is a real nice basic DataFrame, but it omits the last observations on the page (it misses it). 

Hence, I augment it with the data set based on row finding, which finds it but is a little bit more messy. Then I look up the observations and clean them, and add them to the basic dataset. 

The program `geeqie`

```
sudo apt-get install geeqie
```
helps to find the precise coordinates. See [here](https://askubuntu.com/questions/298877/which-image-viewer-is-able-to-show-coordinates) (View > Pixel Info). 


## Clean Data

- First, we clean the basic dataset of Column Headers and sort the data. 
- Then, we find out which numbers are missing from the list (set difference). We also manually append the last number in the document, 5833. 

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

data = (pd.read_csv("../../data/patent_data/interim_patent_data/tabula-erfindungsprivilegien_without_first_page_basic.csv").
    iloc[:, 1:13])

data = data[data['PrivRegNr'] != "PrivRegNr"]
data['PrivRegNr'] = pd.to_numeric(data['PrivRegNr'])
data.sort_values('PrivRegNr', inplace=True)

data

,PrivRegNr,Name1,Vorname1,Name2,Vorname2,Name3,Vorname3,Titel,Schlagworte,Ort,Jahr,Einreichdatu
0,1,Estler,Anton,NaN,NaN,NaN,NaN,Meine Verfahrungsart aus allen Getreide-Stroh-...,Strohpapier,Wien,1816,18.02.1816
1,2,Ofenheimer,Anton,NaN,NaN,NaN,NaN,Das Gebrüder Ofenheimersche Lack-Lake (roter I...,"Cochenille, Schellack, Lak-Lake, Indigo",Wien,1816,10.06.1816
2,3,Rupprecht,Joseph,NaN,NaN,NaN,NaN,Badeapparat mit Brennstoffeinsparung,Badeofen,Wien,1821,07.04.1821
3,4,Voigtländer,Johann Friedrich,NaN,NaN,NaN,NaN,Beschreibung der periskopischen Gläser und cha...,"Periskopische Gläser, Brille",Wien,1816,06.09.1816
4,5,Strauss,Anton,NaN,NaN,NaN,NaN,Erfindung einer Druckmaschine,Druckmaschine,Wien,1815,02.12.1815
...,...,...,...,...,...,...,...,...,...,...,...,...
5786,5811,Regnier,Joseph Franz,NaN,NaN,NaN,NaN,Beschreibung und Zeichnung ... auf die Cimenta...,"Wagenrad, Spurkranz, Geleise,\nZementieren",Fulnek,1842,23.10.1842
5787,5812,Lorenzi,Giovanni di,NaN,NaN,NaN,NaN,Descrizione del Timpantono ... [Pauke],Pauke,Vicenza,1847,10.07.1847
5788,5813,Ressel,Joseph,NaN,NaN,NaN,NaN,Beschreibung der Zeichnung des neuen Dampfschi...,"Schiffsantrieb, Wasserrad",Triest,1851,07.07.1851
5789,5814,Gleischner,Georg,Hohenblum,Joseph R.v.,NaN,NaN,Beschreibung der Verbesserung in Erzeugung von...,"Schneidewerkzeug, Hobeleisen,\nSchleifscheibe",Rehberg bei\nKrems,1847,18.10.1847


In [2]:
present_elements = set(data['PrivRegNr'])
all_elements = set([i for i in range(1, 5816)] + [5833])

missing_elements = all_elements.difference(present_elements)

# Appending the dataset with the missing elements 

Write a function that takes `missing_elements` as input, then parses each `.csv`, merges the rows from the missing number and the row below per column, and then coalesce the 2 columns next to "Schlagworte" if one of those columns is empty. Then extracts the rows (including column names) for the `missing_elements`. 


In [3]:
def list_files_in_directory(path):
    # List all files in the directory
    return [str(file) for file in Path(path).iterdir() if file.is_file()]
def coalesce_columns(df, col1, col2):
    """
    Merge two columns into one by filling missing values from col1 with values from col2.
    """
    return df[col1].combine_first(df[col2])

def merge_missing_rows(df):
    """
    Merge rows where the 'PrivRegNr' is missing.
    """
    merged_row = df.iloc[0].combine_first(df.iloc[1])
     # Create a new DataFrame with the merged row
    merged_df = pd.DataFrame([merged_row])  # Wrap the Series in a list to create a DataFrame

    return merged_df

def is_column_empty(df, column_idx):
    """
    Check if a column is completely empty (contains no text or all values are NaN).
    """
    # Step 1: Replace empty strings with NaN
    df.iloc[:, column_idx].replace("", np.nan, inplace=True)
    
    # Step 2: Check if all values in the column are NaN
    return df.iloc[:, column_idx].isna().all()

def process_csv(file_path, missing_elements):
    """
    Main function to process the CSV, merge rows with missing elements,
    coalesce Schlagworte columns, and extract rows with missing elements.
    """
    
    out = pd.DataFrame()
    files = list_files_in_directory(file_path)
    for file in files:
        print(file)
        # Load the CSV file into a DataFrame
        df = pd.read_csv(file).reset_index()
        df['PrivRegNr'] = pd.to_numeric(df['PrivRegNr'])
        
        # Filter the observations on the missing elements and the row below
        retain = df[df['PrivRegNr'].isin(missing_elements)]
        
        if not retain.empty:
            first_int_index = retain.index[0]  # Get the first index directly
            second_int_index = first_int_index + 1
            keep = [first_int_index, second_int_index]
            
            try:
                # Always use `df.loc[[index]]` to ensure a DataFrame is returned
                df = df.loc[keep]
            except:
                # Wrap `first_int_index` in a list to ensure it remains a DataFrame
                df = df.loc[[first_int_index]]
        else:
            # Handle case where `retain` is empty
            pass

        # Merge the rows of the retained df
        if df.shape[0] > 1:
            df = merge_missing_rows(df)
            
        # Coalesce columns next to 'Schlagworte' if one of these is empty:
        if "Schlagworte" in df.columns:
            col_idx = df.columns.get_loc("Schlagworte")
        else:
            col_idx = np.where(data.columns.str.contains('Schlag'))[0][0]
        if (is_column_empty(df, col_idx+1) | is_column_empty(df, col_idx+2)):
            df['Ort'] = coalesce_columns(df, df.columns[col_idx+1], df.columns[col_idx+2])
        
        # Append the new line to the already existing df
        out = pd.concat([out, df], ignore_index=True)

    return out

In [4]:
missing_obs = process_csv("../../data/patent_data/interim_patent_data/augment", missing_elements)
print(missing_obs.shape)
print(len(missing_elements))

../../data/patent_data/interim_patent_data/augment/tabula-erfindungsprivilegien_without_first_page-62.csv
../../data/patent_data/interim_patent_data/augment/tabula-erfindungsprivilegien_without_first_page-99.csv
../../data/patent_data/interim_patent_data/augment/tabula-erfindungsprivilegien_without_first_page-143.csv
../../data/patent_data/interim_patent_data/augment/tabula-erfindungsprivilegien_without_first_page-178.csv
../../data/patent_data/interim_patent_data/augment/tabula-erfindungsprivilegien_without_first_page-148.csv
../../data/patent_data/interim_patent_data/augment/tabula-erfindungsprivilegien_without_first_page-106.csv
../../data/patent_data/interim_patent_data/augment/tabula-erfindungsprivilegien_without_first_page-27.csv
../../data/patent_data/interim_patent_data/augment/tabula-erfindungsprivilegien_without_first_page-46.csv
../../data/patent_data/interim_patent_data/augment/tabula-erfindungsprivilegien_without_first_page-90.csv
../../data/patent_data/interim_patent_data

## Cleaning and Adding to Original Dataset

- So we found 227 out of 270 "missing observations". In fact, we found all of them since sometimes the actual database skips some numbers. Now, we only select the wanted variables and add them to the original dataset:

In [5]:
data.shape[0] + missing_obs.shape[0]

5792

In [6]:
cols = [i for i in data.columns]
final_data = pd.concat([data, missing_obs[cols]]).sort_values('PrivRegNr')

final_data['PrivRegNr'] = pd.to_numeric(final_data['PrivRegNr'], downcast='integer')
final_data['Jahr'] = pd.to_numeric(final_data['Jahr'], downcast='integer')
final_data.reset_index().iloc[:, 1:].to_csv("../../data/patent_data/erfindungen_data.csv", index=False, sep=";")

In [7]:
final_data

,PrivRegNr,Name1,Vorname1,Name2,Vorname2,Name3,Vorname3,Titel,Schlagworte,Ort,Jahr,Einreichdatu
0,1,Estler,Anton,NaN,NaN,NaN,NaN,Meine Verfahrungsart aus allen Getreide-Stroh-...,Strohpapier,Wien,1816,18.02.1816
1,2,Ofenheimer,Anton,NaN,NaN,NaN,NaN,Das Gebrüder Ofenheimersche Lack-Lake (roter I...,"Cochenille, Schellack, Lak-Lake, Indigo",Wien,1816,10.06.1816
2,3,Rupprecht,Joseph,NaN,NaN,NaN,NaN,Badeapparat mit Brennstoffeinsparung,Badeofen,Wien,1821,07.04.1821
3,4,Voigtländer,Johann Friedrich,NaN,NaN,NaN,NaN,Beschreibung der periskopischen Gläser und cha...,"Periskopische Gläser, Brille",Wien,1816,06.09.1816
4,5,Strauss,Anton,NaN,NaN,NaN,NaN,Erfindung einer Druckmaschine,Druckmaschine,Wien,1815,02.12.1815
...,...,...,...,...,...,...,...,...,...,...,...,...
5787,5812,Lorenzi,Giovanni di,NaN,NaN,NaN,NaN,Descrizione del Timpantono ... [Pauke],Pauke,Vicenza,1847,10.07.1847
5788,5813,Ressel,Joseph,NaN,NaN,NaN,NaN,Beschreibung der Zeichnung des neuen Dampfschi...,"Schiffsantrieb, Wasserrad",Triest,1851,07.07.1851
5789,5814,Gleischner,Georg,Hohenblum,Joseph R.v.,NaN,NaN,Beschreibung der Verbesserung in Erzeugung von...,"Schneidewerkzeug, Hobeleisen,\nSchleifscheibe",Rehberg bei\nKrems,1847,18.10.1847
5790,5815,Hollub,Eduard,Kober,Susanna,Reitzes,Deborah,Detailirte Beschreibung der ... neuen Sicherhe...,Zigarrenanzünder,Wien,1847,21.02.1847
